<a href="https://colab.research.google.com/github/shahmehul2005/RIS-placement-using-CNN/blob/main/collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls -l /content/drive/MyDrive/

ls: /content/drive/MyDrive/exported_data: No such file or directory
total 11479007
drwx------ 2 root root        4096 Jul 12 12:18 'Colab Notebooks'
lrw------- 1 root root           0 Jul 19 13:43  exported_data -> /content/drive/.shortcut-targets-by-id/1YJLMkIacvVePd6oPzzYNiBMIwveylN_D/exported_data
-rw------- 1 root root 11754498403 Jul 20 07:23  uavid.zip


In [ ]:
!unzip -q /content/drive/MyDrive/uavid.zip

In [ ]:
import glob

# Set your image directory path
image_dir = "/content/uavid_val"  # 👈 Update this path

# Get all image files
image_files = glob.glob(f"{image_dir}/**/*.*", recursive=True)

# Filter for image extensions
valid_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp')
image_files = [f for f in image_files if f.lower().endswith(valid_extensions)]

print(f"Total images: {len(image_files)}")

Total images: 140


In [ ]:
from google.colab import drive
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import random


dataset_base_path = '/content/uavid_train'
output_base_path = '/content/uavid_yolo_labels'

# List of sequence folders to process
sequences_to_process = 'seq1'
sequences = [s.strip() for s in sequences_to_process.split(',')]

# UAVid Color Map
UAVID_COLOR_MAP = {
    'Building': (128, 0, 0),
    'Road': (128, 64, 128),
    'Tree': (0, 128, 0),
    'Low vegetation': (128, 128, 0),
    'Moving car': (64, 0, 128),
    'Static car': (192, 0, 192),
    'Human': (64, 64, 0),
    'Background clutter': (0, 0, 0)
}

# Classes for instance segmentation
INSTANCE_CLASSES = ['Building', 'Tree', 'Low vegetation']
CLASS_ID_MAP = {cls: idx for idx, cls in enumerate(INSTANCE_CLASSES)}

def apply_watershed_to_class(class_mask, original_image):
    """Apply watershed algorithm to a binary class mask"""
    kernel = np.ones((3, 3), np.uint8)
    opening = cv2.morphologyEx(class_mask, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.5 * dist_transform.max(), 255, 0)

    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)

    _, markers = cv2.connectedComponents(sure_fg)
    markers = markers + 1
    markers[unknown == 255] = 0

    markers = cv2.watershed(original_image, markers)
    markers[markers == 1] = 0
    markers[markers == -1] = 0

    return markers

def get_yolo_annotations(semantic_img_rgb, original_img):
    """Generate YOLO format annotations from semantic segmentation"""
    annotations = []
    height, width, _ = original_img.shape

    for class_name in INSTANCE_CLASSES:
        color = UAVID_COLOR_MAP[class_name]
        mask = cv2.inRange(semantic_img_rgb, color, color)

        if np.any(mask):
            instance_labels = apply_watershed_to_class(mask, original_img)
            unique_ids = np.unique(instance_labels)
            unique_ids = unique_ids[unique_ids != 0]

            for uid in unique_ids:
                instance_mask = (instance_labels == uid)
                if np.count_nonzero(instance_mask) < 10:  # Skip small instances
                    continue

                coords = np.where(instance_mask)
                y_min, y_max = np.min(coords[0]), np.max(coords[0])
                x_min, x_max = np.min(coords[1]), np.max(coords[1])

                # Calculate YOLO format (normalized center coordinates and dimensions)
                center_x = (x_min + x_max) / 2.0 / width
                center_y = (y_min + y_max) / 2.0 / height
                bbox_width = (x_max - x_min) / width
                bbox_height = (y_max - y_min) / height

                # Clip values to [0,1] range
                center_x = np.clip(center_x, 0, 1)
                center_y = np.clip(center_y, 0, 1)
                bbox_width = np.clip(bbox_width, 0, 1)
                bbox_height = np.clip(bbox_height, 0, 1)

                class_id = CLASS_ID_MAP[class_name]
                annotations.append((class_id, center_x, center_y, bbox_width, bbox_height))

    return annotations

print("Starting conversion to YOLO format...")
print(f"Processing sequences: {sequences}")
print(f"Input path: {dataset_base_path}")
print(f"Output path: {output_base_path}")

visualization_data = []

for seq in sequences:
    seq_path = os.path.join(dataset_base_path, seq)
    image_folder = os.path.join(seq_path, 'Images')
    label_folder = os.path.join(seq_path, 'Labels')
    output_seq_path = os.path.join(output_base_path, seq, 'labels')
    os.makedirs(output_seq_path, exist_ok=True)

    if not os.path.isdir(image_folder) or not os.path.isdir(label_folder):
        print(f"Warning: Missing folders in {seq_path}. Skipping.")
        continue

    print(f"\nProcessing sequence: {seq}")
    label_files = sorted([f for f in os.listdir(label_folder) if f.endswith('.png')])

    for i, label_file in enumerate(label_files):
        label_path = os.path.join(label_folder, label_file)
        image_path = os.path.join(image_folder, label_file.replace('.png', '.jpg'))

        if not os.path.exists(image_path):
            image_path = image_path.replace('.jpg', '.png')
            if not os.path.exists(image_path):
                print(f"  - Image not found for {label_file}. Skipping.")
                continue

        # Read and process images
        semantic_img = cv2.imread(label_path)
        if semantic_img is None:
            print(f"  - Could not read semantic label: {label_path}")
            continue

        semantic_img_rgb = cv2.cvtColor(semantic_img, cv2.COLOR_BGR2RGB)
        original_img = cv2.imread(image_path)
        if original_img is None:
            print(f"  - Could not read image: {image_path}")
            continue

        # Get YOLO annotations
        annotations = get_yolo_annotations(semantic_img_rgb, original_img)

        # Save to text file
        txt_filename = label_file.replace('.png', '.txt')
        txt_path = os.path.join(output_seq_path, txt_filename)
        with open(txt_path, 'w') as f:
            for ann in annotations:
                f.write(f"{ann[0]} {ann[1]:.6f} {ann[2]:.6f} {ann[3]:.6f} {ann[4]:.6f}\n")

        # Store first 3 for visualization
        if i < 3:
            visualization_data.append({
                'image_path': image_path,
                'txt_path': txt_path,
                'semantic_path': label_path
            })

print("\nConversion complete!")

# Visualization
if visualization_data:
    print("\nVisualizing results...")
    for data in visualization_data:
        img = cv2.imread(data['image_path'])
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width, _ = img.shape

        # Read annotations
        with open(data['txt_path']) as f:
            annotations = []
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    class_id, cx, cy, w, h = map(float, parts)
                    annotations.append((int(class_id), cx, cy, w, h))

        # Draw bounding boxes
        img_with_boxes = img_rgb.copy()
        for ann in annotations:
            class_id, cx, cy, w, h = ann
            # Convert from normalized to pixel coordinates
            x_center = int(cx * width)
            y_center = int(cy * height)
            bbox_width = int(w * width)
            bbox_height = int(h * height)

            x_min = max(0, x_center - bbox_width // 2)
            y_min = max(0, y_center - bbox_height // 2)
            x_max = min(width, x_center + bbox_width // 2)
            y_max = min(height, y_center + bbox_height // 2)

            # Draw rectangle and label
            color = (0, 255, 0)  # Green
            cv2.rectangle(img_with_boxes, (x_min, y_min), (x_max, y_max), color, 2)
            class_name = INSTANCE_CLASSES[class_id]
            cv2.putText(img_with_boxes, class_name, (x_min, y_min - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Display results
        fig, ax = plt.subplots(1, 2, figsize=(20, 10))
        ax[0].imshow(img_rgb)
        ax[0].set_title('Original Image')
        ax[0].axis('off')

        ax[1].imshow(img_with_boxes)
        ax[1].set_title('YOLO Annotations')
        ax[1].axis('off')

        plt.tight_layout()
        plt.show()
else:
    print("\nNo files processed for visualization")

In [ ]:
!cp -r /content/uavid_yolo_labels2 /content/drive/MyDrive/

In [ ]:
##

In [ ]:
from google.colab import files
files.upload()  # Upload the downloaded `kaggle.json`


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shahmehul5002","key":"9b6639e7976cd03819b326ab5c16a2f8"}'}

In [ ]:
!pip install -q kaggle  # Install Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set correct permissions

In [ ]:
!kaggle datasets download kushagrapandya/visdrone-dataset

Dataset URL: https://www.kaggle.com/datasets/kushagrapandya/visdrone-dataset
License(s): GNU Affero General Public License 3.0
100% 2.09G/2.10G [00:24<00:00, 242MB/s]
100% 2.10G/2.10G [00:24<00:00, 90.4MB/s]


In [ ]:
from google.colab import drive
# 1. Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# List shared folders
!ls -l "/content/drive/MyDrive"

ls: /content/drive/MyDrive/exported_data: No such file or directory
ls: /content/drive/MyDrive/uavid_yolo_labels: No such file or directory
total 96149
-rw------- 1 root root 39478065 Jul 11 11:50  abc.tif
drwx------ 2 root root     4096 Jul  5 09:09 'Colab Notebooks'
-rw------- 1 root root 52079139 Jul 12 05:35  drone_deploy.pt
lrw------- 1 root root        0 Jul 20 06:54  exported_data -> /content/drive/.shortcut-targets-by-id/1YJLMkIacvVePd6oPzzYNiBMIwveylN_D/exported_data
-rw------- 1 root root  6889997 Jul 12 05:40  P0054.png
lrw------- 1 root root        0 Jul 24 06:32  uavid_yolo_labels -> /content/drive/.shortcut-targets-by-id/1rl-WOOOuvuvXmciZUPLst2InKEsQItXg/uavid_yolo_labels
drwx------ 2 root root     4096 Jul  9 12:40  yolo_training


In [ ]:
!cp -r "/content/drive/MyDrive/uavid_yolo_labels" "/content/"

In [ ]:
!cp -r "/content/drive/MyDrive/yolo_training/uavid.zip" "/content/"

In [ ]:
!unzip -q uavid.zip

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Config
image_dir = "/content/uavid_train/seq1/Images"      # Original image directory
label_dir = "/content/uavid_yolo_labels/seq1/labels"      # Original label directory
dataset_path = "my_dataset"            # Output directory
val_ratio = 0.2                        # 80% train, 20% validation

# Create folders
os.makedirs(f"{dataset_path}/images/train", exist_ok=True)
os.makedirs(f"{dataset_path}/images/val", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/val", exist_ok=True)

# Get image filenames (without extensions)
image_files = [f.split('.')[0] for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
image_files_train, image_files_val = train_test_split(image_files, test_size=val_ratio, random_state=42)

# Helper function to copy files
def copy_files(filenames, source_dir, dest_dir_img, dest_dir_label, ext=".png"):
    for f in filenames:
        # Copy image
        shutil.copy2(
            os.path.join(source_dir, f + ext),
            os.path.join(dest_dir_img, f + ext)
        )
        # Copy label
        label_file = f + ".txt"
        shutil.copy2(
            os.path.join(label_dir, label_file),
            os.path.join(dest_dir_label, label_file)
        )

# Copy train/val data
copy_files(image_files_train, image_dir, f"{dataset_path}/images/train", f"{dataset_path}/labels/train")
copy_files(image_files_val, image_dir, f"{dataset_path}/images/val", f"{dataset_path}/labels/val")

print(f"Train: {len(image_files_train)} images | Val: {len(image_files_val)} images")

Train: 480 images | Val: 120 images


In [ ]:
!unzip -q /content/visdrone-dataset.zip

In [ ]:


# --- STEP 1: TRAIN THE TEACHER MODEL ---
print("--- Starting Teacher Model Training ---")

# Initialize a YOLO model (using pre-trained weights)
teacher_model = YOLO(TEACHER_MODEL_PATH)

# Train the teacher model on the labeled dataset
teacher_model.train(
    data=LABELED_DATA_YAML,
    epochs=TEACHER_EPOCHS,
    imgsz=TEACHER_IMG_SIZE,
    batch=TEACHER_BATCH_SIZE,
    project=TEACHER_PROJECT_NAME,
    name='teacher_run',
    patience=20
)






--- Starting Teacher Model Training ---
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/my_dataset/student_data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=teacher_run9, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience

100%|██████████| 755k/755k [00:00<00:00, 27.6MB/s]

Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,433 parameters, 3,011,417 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 82.0MB/s]


...

                   all        120       2175      0.631      0.439      0.476      0.263



50 epochs completed in 2.143 hours.
Optimizer stripped from teacher_training/teacher_run9/weights/last.pt, 6.3MB
Optimizer stripped from teacher_training/teacher_run9/weights/best.pt, 6.3MB

Validating teacher_training/teacher_run9/weights/best.pt...
Ultralytics 8.3.169 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:32<00:00,  4.07s/it]


                   all        120       2175      0.629      0.438      0.475      0.263
              building        119        562      0.679      0.537       0.57      0.338
                  tree        120        896      0.615      0.432      0.471      0.243
        low vegetation        120        717      0.592      0.346      0.385      0.207
Speed: 0.6ms preprocess, 6.1ms inference, 0.0ms loss, 5.7ms postprocess per image
Results saved to teacher_training/teacher_run9


FileNotFoundError: [Errno 2] No such file or directory: 'teacher_project/teacher_run/weights/best.pt'

In [ ]:
import os
import shutil

# 1. Define paths
TEACHER_PROJECT_NAME = "teacher_training"  # Replace with your actual project name
best_teacher_weights = os.path.join(TEACHER_PROJECT_NAME, 'teacher_run9/weights/best.pt')

# 2. Configure export directory (customize this)
drive_export_path = '/content/drive/MyDrive/models/yolo_teacher'
os.makedirs(drive_export_path, exist_ok=True)

# 3. Export model
drive_model_path = os.path.join(drive_export_path, 'teacher_best.pt')
shutil.copy(best_teacher_weights, drive_model_path)

print("--- Teacher Model Exported to Google Drive ---")
print(f"Source: {best_teacher_weights}")
print(f"Destination: {drive_model_path}")

--- Teacher Model Exported to Google Drive ---
Source: teacher_training/teacher_run9/weights/best.pt
Destination: /content/drive/MyDrive/models/yolo_teacher/teacher_best.pt


In [ ]:
##

In [ ]:
from google.colab import files
files.upload()  # Upload the downloaded `kaggle.json`


Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"shahmehul5002","key":"9b6639e7976cd03819b326ab5c16a2f8"}'}

In [ ]:
!pip install -q kaggle  # Install Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set correct permissions

In [ ]:
!kaggle datasets download kushagrapandya/visdrone-dataset

Dataset URL: https://www.kaggle.com/datasets/kushagrapandya/visdrone-dataset
License(s): GNU Affero General Public License 3.0
 99% 2.07G/2.10G [00:12<00:00, 169MB/s]
100% 2.10G/2.10G [00:12<00:00, 178MB/s]


In [ ]:
from google.colab import drive
# 1. Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# List shared folders
!ls -l "/content/drive/MyDrive"

ls: /content/drive/MyDrive/uavid_yolo_labels: No such file or directory
total 11485175
drwx------ 2 root root        4096 Jul 15 03:30 'Colab Notebooks'
drwx------ 2 root root        4096 Jul 19 14:11  exported_data
-rw------- 1 root root     6312355 Jul 25 04:24  teacher_best.pt
lrw------- 1 root root           0 Jul 25 04:25  uavid_yolo_labels -> /content/drive/.shortcut-targets-by-id/1rl-WOOOuvuvXmciZUPLst2InKEsQItXg/uavid_yolo_labels
-rw------- 1 root root 11754498403 Jul 25 04:24  uavid.zip


In [ ]:
!cp -r "/content/drive/MyDrive/uavid.zip" "/content/"

In [ ]:
!cp -r "/content/drive/MyDrive/uavid_yolo_labels" "/content/"

In [ ]:
!unzip -q uavid.zip

In [ ]:
!unzip -q /content/visdrone-dataset.zip

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Config
image_dir = "/content/uavid_train/seq1/Images"      # Original image directory
label_dir = "/content/uavid_yolo_labels/seq1/labels"      # Original label directory
dataset_path = "my_dataset"            # Output directory
val_ratio = 0.2                        # 80% train, 20% validation

# Create folders
os.makedirs(f"{dataset_path}/images/train", exist_ok=True)
os.makedirs(f"{dataset_path}/images/val", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/train", exist_ok=True)
os.makedirs(f"{dataset_path}/labels/val", exist_ok=True)

# Get image filenames (without extensions)
image_files = [f.split('.')[0] for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
image_files_train, image_files_val = train_test_split(image_files, test_size=val_ratio, random_state=42)

# Helper function to copy files
def copy_files(filenames, source_dir, dest_dir_img, dest_dir_label, ext=".png"):
    for f in filenames:
        # Copy image
        shutil.copy2(
            os.path.join(source_dir, f + ext),
            os.path.join(dest_dir_img, f + ext)
        )
        # Copy label
        label_file = f + ".txt"
        shutil.copy2(
            os.path.join(label_dir, label_file),
            os.path.join(dest_dir_label, label_file)
        )

# Copy train/val data
copy_files(image_files_train, image_dir, f"{dataset_path}/images/train", f"{dataset_path}/labels/train")
copy_files(image_files_val, image_dir, f"{dataset_path}/images/val", f"{dataset_path}/labels/val")

print(f"Train: {len(image_files_train)} images | Val: {len(image_files_val)} images")

Train: 480 images | Val: 120 images


In [ ]:
import os
import shutil
from ultralytics import YOLO
import cv2

# --- CONFIGURATION ---

# 1. Paths
LABELED_DATA_YAML = os.path.join('/content/my_dataset', 'student_data.yaml')
with open(LABELED_DATA_YAML, 'w') as f:
    f.write(f"path: {os.path.abspath('/content/my_dataset')}\n")
    f.write("train: images/train\n")
    # You should copy your original validation set for a proper evaluation
    f.write("val: images/val\n\n")
    f.write("nc: 3\n")
    f.write("names:")
    # Add your class names here, matching the original data.yaml
    f.write(" ['building' , 'tree' , 'low vegetation']")
UNLABELED_IMAGES_DIR = '/content/VisDrone2019-DET-train/VisDrone2019-DET-train/images'
COMBINED_DATASET_DIR = 'combined_dataset/'

# 2. Teacher Model Training Parameters
TEACHER_MODEL_PATH = 'yolov8n.pt'  # Start with pre-trained weights
TEACHER_EPOCHS = 50
TEACHER_IMG_SIZE = 1280
TEACHER_BATCH_SIZE = 8
TEACHER_PROJECT_NAME = 'teacher_training'

# 3. Pseudo-Labeling Parameters
CONFIDENCE_THRESHOLD = 0.85  # Only save predictions with confidence > 85%

# 4. Student Model Training Parameters
STUDENT_MODEL_PATH = 'yolov8n.pt' # Start student from the same pre-trained weights
STUDENT_EPOCHS = 100 # Train for more epochs on the larger dataset
STUDENT_IMG_SIZE = 1280
STUDENT_BATCH_SIZE = 8
STUDENT_PROJECT_NAME = 'student_training'

# --- STEP 2: GENERATE AND FILTER PSEUDO-LABELS ---
print(f"--- Generating Pseudo-Labels with Confidence Threshold > {CONFIDENCE_THRESHOLD} ---")

best_teacher_weights = '/content/drive/MyDrive/teacher_best.pt'
# Load the trained teacher model
teacher_model = YOLO(best_teacher_weights)

# Create a directory to store the pseudo-labels
pseudo_labels_dir = os.path.join(COMBINED_DATASET_DIR, 'train/labels/')
os.makedirs(pseudo_labels_dir, exist_ok=True)

# Get list of unlabeled images
unlabeled_images = os.listdir(UNLABELED_IMAGES_DIR)

for image_name in unlabeled_images:
    image_path = os.path.join(UNLABELED_IMAGES_DIR, image_name)

    # Run inference
    results = teacher_model.predict(image_path, verbose=False)

    # Get the base name of the image file (without extension)
    base_name = os.path.splitext(image_name)
    label_path = os.path.join(pseudo_labels_dir, f'{base_name}.txt')

    # Save high-confidence predictions to a YOLO format.txt file
    with open(label_path, 'w') as f:
      if results[0].boxes is not None:
        for box in results[0].boxes:
            class_id = int(box.cls.item())  # Added .item() to get scalar value

            # Extract coordinates properly
            coords = box.xywhn[0]  # Get first (and only) element of batch
            x_center = coords[0].item()
            y_center = coords[1].item()
            width = coords[2].item()
            height = coords[3].item()
            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

print(f"--- Pseudo-Label Generation Complete. Labels saved in {pseudo_labels_dir} ---")


# --- STEP 3: COMBINE THE DATASETS ---
print("--- Combining Labeled and Pseudo-Labeled Datasets ---")

# Create directory structure for the combined dataset
combined_images_dir = os.path.join(COMBINED_DATASET_DIR, 'train/images/')
os.makedirs(combined_images_dir, exist_ok=True)

# a) Copy original labeled images and labels
# Assuming the original labeled dataset is in 'labeled_dataset/train/'
original_images_dir = '/content/my_dataset/images/train'
original_labels_dir = '/content/my_dataset/labels/train'

for filename in os.listdir(original_images_dir):
    shutil.copy(os.path.join(original_images_dir, filename), combined_images_dir)
for filename in os.listdir(original_labels_dir):
    shutil.copy(os.path.join(original_labels_dir, filename), pseudo_labels_dir)

# b) Copy unlabeled images (which now have pseudo-labels)
for filename in os.listdir(UNLABELED_IMAGES_DIR):
    shutil.copy(os.path.join(UNLABELED_IMAGES_DIR, filename), combined_images_dir)




--- Generating Pseudo-Labels with Confidence Threshold > 0.85 ---
--- Pseudo-Label Generation Complete. Labels saved in combined_dataset/train/labels/ ---
--- Combining Labeled and Pseudo-Labeled Datasets ---


In [ ]:
# Dataset paths
image_dir = '/content/combined_dataset/train/images'
label_dir = '/content/combined_dataset/train/labels'

# Class mapping (model class ID -> target class ID in your dataset)
class_mapping = {
    0: 3,
    1: 4# Model's class 0 becomes class 5 in your labels
}

conf_threshold = 0.1  # Minimum confidence score

In [ ]:
!pip install -q transformers timm torch opencv-python supervision

import cv2
import torch
import numpy as np
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
import supervision as sv
import os
import glob

# Use the standard OWL-ViT model from Google
model_name = "google/owlvit-base-patch32"
processor = AutoProcessor.from_pretrained(model_name)
model = AutoModelForZeroShotObjectDetection.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 6.2 MB/s eta 0:00:00


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

OwlViTForObjectDetection(
  (owlvit): OwlViTModel(
    (text_model): OwlViTTextTransformer(
      (embeddings): OwlViTTextEmbeddings(
        (token_embedding): Embedding(49408, 512)
        (position_embedding): Embedding(16, 512)
      )
      (encoder): OwlViTEncoder(
        (layers): ModuleList(
          (0-11): 12 x OwlViTEncoderLayer(
            (self_attn): OwlViTAttention(
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (out_proj): Linear(in_features=512, out_features=512, bias=True)
            )
            (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
            (mlp): OwlViTMLP(
              (activation_fn): QuickGELUActivation()
              (fc1): Linear(in_features=512, out_features=2048, bias=True)
              (fc2): Linear(in_features=2048, out_

In [ ]:
text_queries = ["transmission tower", "communication tower"]
# Get all image paths (both JPG and PNG)
image_paths = []
for ext in ('*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG'):
    image_paths.extend(glob.glob(os.path.join(image_dir, ext)))

for img_path in image_paths:
    # Get corresponding label path
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(label_dir, f"{base_name}.txt")

    # Read existing labels (if any)
    existing_boxes = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            existing_boxes = f.readlines()

    # Load image with PIL (required by OWL-ViT)
    image = Image.open(img_path).convert("RGB")
    width, height = image.size

    # Prepare inputs for OWL-ViT
    inputs = processor(
        text=text_queries,
        images=image,
        return_tensors="pt"
    ).to(device)

    # Run inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Process predictions
    target_sizes = torch.tensor([image.size[::-1]])
    results = processor.post_process_grounded_object_detection(
        outputs=outputs,
        threshold=conf_threshold,
        target_sizes=target_sizes
    )[0]

    # Extract detections
    new_boxes = []
    for box, score, label in zip(results["boxes"], results["scores"], results["labels"]):
        # Skip if label not in mapping
        if label.item() not in class_mapping:
            continue

        # Convert to normalized YOLO format
        x_min, y_min, x_max, y_max = box.tolist()

        # Normalize coordinates
        x_center = (x_min + x_max) / 2 / width
        y_center = (y_min + y_max) / 2 / height
        w = (x_max - x_min) / width
        h = (y_max - y_min) / height

        # Format: class_id x_center y_center width height
        new_boxes.append(
            f"{class_mapping[label.item()]} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n"
        )

    # Write back original labels + new detections
    with open(label_path, 'w') as f:
        # Write existing boxes first
        f.writelines(existing_boxes)
        # Append new boxes
        f.writelines(new_boxes)

In [ ]:
!cp -r /content/combined_dataset /content/drive/MyDrive

In [ ]:
##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/drive/MyDrive/combined_dataset /content

In [ ]:
!cp -r /content/combined_dataset /content/data1

In [ ]:
!cp -r /content/combined_dataset /content/data2

In [ ]:
import os
import shutil
from tqdm import tqdm

# Set your paths here
IMAGES_DIR = "/content/data1/train/images"  # Path to your images folder
LABELS_DIR = "/content/data1/train/labels"  # Path to your labels folder
BACKUP_DIR = "/content/backup"  # Backup directory for removed files

# Create backup directories
os.makedirs(os.path.join(BACKUP_DIR, "images"), exist_ok=True)
os.makedirs(os.path.join(BACKUP_DIR, "labels"), exist_ok=True)

# ========================
# SOLUTION 1: Remove empty or only low-vegetation annotations
# ========================
def remove_empty_or_low_veg(images_dir, labels_dir, backup_dir):
    """Remove images/labels that are empty or contain only low vegetation (class 2)"""
    removed_count = 0
    for label_file in tqdm(os.listdir(labels_dir), desc="Processing Solution 1"):
        label_path = os.path.join(labels_dir, label_file)
        image_file = os.path.splitext(label_file)[0]  # Remove .txt extension

        # Find matching image with any extension
        image_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            possible_path = os.path.join(images_dir, image_file + ext)
            if os.path.exists(possible_path):
                image_path = possible_path
                break

        if not image_path:
            continue

        # Check label content
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Get unique classes in the file
        classes = set()
        for line in lines:
            if line.strip():  # Skip empty lines
                try:
                    cls_id = int(line.split()[0])
                    classes.add(cls_id)
                except:
                    continue

        # Remove if: empty or contains only class 2 (low vegetation)
        if not classes or classes == {2}:
            # Move files to backup
            shutil.move(image_path, os.path.join(backup_dir, "images", os.path.basename(image_path)))
            shutil.move(label_path, os.path.join(backup_dir, "labels", label_file))
            removed_count += 1

    print(f"Solution 1: Removed {removed_count} images with only low vegetation or empty labels")

# ========================
# SOLUTION 2: Remove images with <3 annotations
# ========================
def remove_few_annotations(images_dir, labels_dir, backup_dir, min_annotations=3):
    """Remove images with fewer than min_annotations"""
    removed_count = 0
    for label_file in tqdm(os.listdir(labels_dir), desc="Processing Solution 2"):
        label_path = os.path.join(labels_dir, label_file)
        image_file = os.path.splitext(label_file)[0]  # Remove .txt extension

        # Find matching image with any extension
        image_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            possible_path = os.path.join(images_dir, image_file + ext)
            if os.path.exists(possible_path):
                image_path = possible_path
                break

        if not image_path:
            continue

        # Count valid annotations
        with open(label_path, 'r') as f:
            lines = f.readlines()

        valid_annotations = 0
        for line in lines:
            if line.strip():  # Count non-empty lines
                valid_annotations += 1

        # Remove if doesn't meet minimum annotations
        if valid_annotations < min_annotations:
            # Move files to backup
            shutil.move(image_path, os.path.join(backup_dir, "images", os.path.basename(image_path)))
            shutil.move(label_path, os.path.join(backup_dir, "labels", label_file))
            removed_count += 1

    print(f"Solution 2: Removed {removed_count} images with fewer than {min_annotations} annotations")

# ========================
# EXECUTION
# ========================
# Run Solution 1
remove_empty_or_low_veg(IMAGES_DIR, LABELS_DIR, BACKUP_DIR)

# Run Solution 2
#remove_few_annotations(IMAGES_DIR, LABELS_DIR, BACKUP_DIR, min_annotations=3)

print("Dataset reduction complete!")
print(f"Removed files are backed up at: {BACKUP_DIR}")

Processing Solution 1: 100%|██████████| 6951/6951 [10:18<00:00, 11.24it/s] 

Solution 1: Removed 780 images with only low vegetation or empty labels
Dataset reduction complete!
Removed files are backed up at: /content/backup


In [ ]:
import os
import shutil
from tqdm import tqdm

# Set your paths here
IMAGES_DIR = "/content/data2/train/images"  # Path to your images folder
LABELS_DIR = "/content/data2/train/labels"  # Path to your labels folder
BACKUP_DIR = "/content/backup2"  # Backup directory for removed files

# Create backup directories
os.makedirs(os.path.join(BACKUP_DIR, "images"), exist_ok=True)
os.makedirs(os.path.join(BACKUP_DIR, "labels"), exist_ok=True)

# ========================
# SOLUTION 1: Remove empty or only low-vegetation annotations
# ========================
def remove_empty_or_low_veg(images_dir, labels_dir, backup_dir):
    """Remove images/labels that are empty or contain only low vegetation (class 2)"""
    removed_count = 0
    for label_file in tqdm(os.listdir(labels_dir), desc="Processing Solution 1"):
        label_path = os.path.join(labels_dir, label_file)
        image_file = os.path.splitext(label_file)[0]  # Remove .txt extension

        # Find matching image with any extension
        image_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            possible_path = os.path.join(images_dir, image_file + ext)
            if os.path.exists(possible_path):
                image_path = possible_path
                break

        if not image_path:
            continue

        # Check label content
        with open(label_path, 'r') as f:
            lines = f.readlines()

        # Get unique classes in the file
        classes = set()
        for line in lines:
            if line.strip():  # Skip empty lines
                try:
                    cls_id = int(line.split()[0])
                    classes.add(cls_id)
                except:
                    continue

        # Remove if: empty or contains only class 2 (low vegetation)
        if not classes or classes == {2}:
            # Move files to backup
            shutil.move(image_path, os.path.join(backup_dir, "images", os.path.basename(image_path)))
            shutil.move(label_path, os.path.join(backup_dir, "labels", label_file))
            removed_count += 1

    print(f"Solution 1: Removed {removed_count} images with only low vegetation or empty labels")

# ========================
# SOLUTION 2: Remove images with <3 annotations
# ========================
def remove_few_annotations(images_dir, labels_dir, backup_dir, min_annotations=3):
    """Remove images with fewer than min_annotations"""
    removed_count = 0
    for label_file in tqdm(os.listdir(labels_dir), desc="Processing Solution 2"):
        label_path = os.path.join(labels_dir, label_file)
        image_file = os.path.splitext(label_file)[0]  # Remove .txt extension

        # Find matching image with any extension
        image_path = None
        for ext in ['.jpg', '.jpeg', '.png', '.bmp']:
            possible_path = os.path.join(images_dir, image_file + ext)
            if os.path.exists(possible_path):
                image_path = possible_path
                break

        if not image_path:
            continue

        # Count valid annotations
        with open(label_path, 'r') as f:
            lines = f.readlines()

        valid_annotations = 0
        for line in lines:
            if line.strip():  # Count non-empty lines
                valid_annotations += 1

        # Remove if doesn't meet minimum annotations
        if valid_annotations < min_annotations:
            # Move files to backup
            shutil.move(image_path, os.path.join(backup_dir, "images", os.path.basename(image_path)))
            shutil.move(label_path, os.path.join(backup_dir, "labels", label_file))
            removed_count += 1

    print(f"Solution 2: Removed {removed_count} images with fewer than {min_annotations} annotations")

# ========================
# EXECUTION
# ========================
# Run Solution 1
#remove_empty_or_low_veg(IMAGES_DIR, LABELS_DIR, BACKUP_DIR)

# Run Solution 2
remove_few_annotations(IMAGES_DIR, LABELS_DIR, BACKUP_DIR, min_annotations=3)

print("Dataset reduction complete!")
print(f"Removed files are backed up at: {BACKUP_DIR}")

Processing Solution 2: 100%|██████████| 6171/6171 [11:51<00:00,  8.67it/s]

Solution 2: Removed 1523 images with fewer than 3 annotations
Dataset reduction complete!
Removed files are backed up at: /content/backup


In [ ]:
!cp -r /content/data2 /content/drive/MyDrive

In [ ]:
##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import yaml

def yolo_train_val_split(dataset_path, output_path, train_ratio=0.8, seed=42):
    """
    Split YOLO formatted dataset into train/validation sets

    Args:
        dataset_path: Path to dataset (should contain 'images' and 'labels' folders)
        output_path: Where to save the split datasets
        train_ratio: Percentage of data for training (0-1)
        seed: Random seed for reproducibility
    """
    # Create directories
    os.makedirs(os.path.join(output_path, 'train/images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'train/labels'), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'val/images'), exist_ok=True)
    os.makedirs(os.path.join(output_path, 'val/labels'), exist_ok=True)

    # Get list of image files (supporting both .jpg and .png)
    images_dir = os.path.join(dataset_path, 'images')
    labels_dir = os.path.join(dataset_path, 'labels')

    # Supported image extensions
    image_extensions = ('.jpg', '.jpeg', '.png')
    image_files = [f for f in os.listdir(images_dir)
                  if f.lower().endswith(image_extensions)]

    # Split into train and val sets
    train_files, val_files = train_test_split(
        image_files,
        train_size=train_ratio,
        random_state=seed
    )

    # Helper function to copy files
    def copy_files(files, split_type):
        for file in files:
            # Copy image
            shutil.copy(
                os.path.join(images_dir, file),
                os.path.join(output_path, split_type, 'images', file)
            )

            # Copy corresponding label file
            base_name = os.path.splitext(file)[0]  # Get filename without extension
            possible_label_files = [
                f"{base_name}.txt",                  # Standard YOLO format
                f"{base_name}.TXT",                  # Some systems may capitalize
                f"{base_name}.text"                 # Rare but possible
            ]

            for label_file in possible_label_files:
                src_label = os.path.join(labels_dir, label_file)
                if os.path.exists(src_label):
                    shutil.copy(
                        src_label,
                        os.path.join(output_path, split_type, 'labels', f"{base_name}.txt")
                    )
                    break
            else:
                print(f"Warning: No label file found for {file}")

    # Perform the copying
    copy_files(train_files, 'train')
    copy_files(val_files, 'val')

    print(f"\nSplit completed:\n"
          f"Total images: {len(image_files)}\n"
          f"Training samples: {len(train_files)} ({len(train_files)/len(image_files):.1%})\n"
          f"Validation samples: {len(val_files)} ({len(val_files)/len(image_files):.1%})\n")

    # Create data.yaml file
    class_names = ['building', 'tree', 'low vegetation', 'transmission tower', 'communication tower']
    yaml_content = {
        'path': output_path,
        'train': 'train/images',
        'val': 'val/images',
        'names': {i: name for i, name in enumerate(class_names)},
        'nc': len(class_names)
    }

    with open(os.path.join(output_path, 'data.yaml'), 'w') as f:
        yaml.dump(yaml_content, f, sort_keys=False)

    print(f"Created data.yaml configuration file at: {os.path.join(output_path, 'data.yaml')}")
    print("\nFinal directory structure:")
    print(f"{output_path}")
    print("├── train/")
    print("│   ├── images/")
    print("│   └── labels/")
    print("├── val/")
    print("│   ├── images/")
    print("│   └── labels/")
    print("└── data.yaml")

# Example usage (for Google Colab):
if __name__ == "__main__":
    # Mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Set your paths (modify these)
    raw_dataset_path = "/content/drive/MyDrive/data2/train"  # Contains images/ and labels/
    output_path = "/content/drive/MyDrive/split"    # Where to save split dataset

    # Perform the split (80% train, 20% val)
    yolo_train_val_split(
        dataset_path=raw_dataset_path,
        output_path=output_path,
        train_ratio=0.8,
        seed=42
    )

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Split completed:
Total images: 4648
Training samples: 3718 (80.0%)
Validation samples: 930 (20.0%)

Created data.yaml configuration file at: /content/drive/MyDrive/split/data.yaml

Final directory structure:
/content/drive/MyDrive/split
├── train/
│   ├── images/
│   └── labels/
├── val/
│   ├── images/
│   └── labels/
└── data.yaml


In [ ]:
import os
from pathlib import Path
from google.colab import drive

# Mount Google Drive (if needed)
drive.mount('/content/drive')

# Set your image and label directories
images_dir = Path('/content/drive/MyDrive/split/val/images')  # 👈 UPDATE TO YOUR IMAGES PATH
labels_dir = Path('/content/drive/MyDrive/split/val/labels')  # 👈 UPDATE TO YOUR LABELS PATH

# Supported image extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']

# Create lists of files
image_files = [f for f in images_dir.iterdir() if f.suffix.lower() in image_extensions]
label_files = [f for f in labels_dir.iterdir() if f.suffix.lower() == '.txt']

# Convert to sets for faster lookup
image_stems = {f.stem for f in image_files}
label_stems = {f.stem for f in label_files}

# Find missing labels
images_without_labels = []
for img_stem in image_stems:
    if img_stem not in label_stems:
        # Find the full filename of the image without label
        for img_file in image_files:
            if img_file.stem == img_stem:
                images_without_labels.append(img_file.name)
                break

# Find orphan labels (optional)
orphan_labels = [f.name for f in label_files if f.stem not in image_stems]

# Display results
print(f"Total images: {len(image_files)}")
print(f"Total labels: {len(label_files)}")
print(f"\nImages without labels: {len(images_without_labels)}")
print("-----------------------------------")

if images_without_labels:
    print("Missing label files for these images:")
    for img in images_without_labels[:20]:  # Show first 20 to avoid flooding output
        print(f" - {img}")
    if len(images_without_labels) > 20:
        print(f"... and {len(images_without_labels) - 20} more")
else:
    print("✅ All images have corresponding labels!")

print(f"\nOrphan labels (without images): {len(orphan_labels)}")
if orphan_labels:
    print("Consider deleting these label files:")
    for lbl in orphan_labels[:20]:  # Show first 20
        print(f" - {lbl}")
    if len(orphan_labels) > 20:
        print(f"... and {len(orphan_labels) - 20} more")

# Summary statistics
print("\n--- Validation Summary ---")
print(f"Images with labels: {len(image_files) - len(images_without_labels)}")
print(f"Images without labels: {len(images_without_labels)}")
print(f"Orphan label files: {len(orphan_labels)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total images: 930
Total labels: 930

Images without labels: 0
-----------------------------------
✅ All images have corresponding labels!

Orphan labels (without images): 0

--- Validation Summary ---
Images with labels: 930
Images without labels: 0
Orphan label files: 0


In [ ]:

!pip install ultralytics pyyaml roboflow --quiet
import os
import yaml
from IPython.display import Image, display


# Verify dataset structure - update these paths to match your dataset location
DATASET_PATH = "/content/drive/MyDrive/split"  # ← CHANGE THIS


dataset_config = {
    'path': DATASET_PATH,
    'train': 'train/images',
    'val': 'valid/images',
    'names': {
        0: 'building',
        1: 'tree',
        2: 'low vegetation',
        3: 'transmission tower',
        4: 'communication tower'
    }
}

# Save the configuration
with open(f'{DATASET_PATH}/data.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

# Verify configuration
print("Dataset configuration:")
!cat '{DATASET_PATH}/data.yaml'

# %%
# Train YOLOv8 model
from ultralytics import YOLO

# Load a pretrained model
model = YOLO('yolov8s.pt')  # Small version (good balance between speed/accuracy)

# Training parameters
results = model.train(
    data=f'{DATASET_PATH}/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    patience=20,
    device=0,  # Use GPU
    pretrained=True,
    save_period=10,
    project='/content/drive/MyDrive/YOLO_Training'
)

# %%
# Validate the model
metrics = model.val()
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")

# %%
# Export to TensorRT (optional)
model.export(format='engine', device=0)  # Requires CUDA

# %%
# Save trained model to Drive
!cp -r '/content/runs/detect/drone_detection' '/content/drive/MyDrive/'
print("Model saved to Google Drive!")

Dataset configuration:
names:
  0: building
  1: tree
  2: low vegetation
  3: transmission tower
  4: communication tower
path: /content/drive/MyDrive/split
train: train/images
val: valid/images
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/split/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, m

Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

 22        [15, 18, 21]  1   2117983  ultralytics.nn.modules.head.Detect           [5, [128, 256, 512]]          
Model summary: 129 layers, 11,137,535 parameters, 11,137,519 gradients, 28.7 GFLOPs

Transferred 349/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.6±0.2 ms, read: 14.2±18.1 MB/s, size: 6226.3 KB)


train: Scanning /content/drive/MyDrive/split/train/labels... 3718 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3718/3718 [20:00<00:00,  3.10it/s]


...

     62/100      4.04G      0.983     0.9054      1.156         44        640: 100%|██████████| 233/233 [03:21<00:00,  1.15it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:32<00:00,  1.08s/it]


                   all        930       6205      0.666      0.363      0.365      0.213

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     63/100      4.04G      0.985      0.898       1.15         71        640: 100%|██████████| 233/233 [03:21<00:00,  1.16it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:30<00:00,  1.01s/it]


                   all        930       6205      0.394      0.385      0.365      0.215

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     64/100      4.04G     0.9768      0.896      1.148        148        640:  45%|████▌     | 105/233 [01:34<01:36,  1.32it/s]

In [ ]:
!cp /content/drive/MyDrive/split/data.yaml /content

In [ ]:
!cp /content/data.yaml /content/drive/MyDrive/split

In [ ]:
##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/high.pt /content

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

In [ ]:
import torch
from PIL import Image
import cv2
import numpy as np

# Load model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/high.pt', force_reload=True)

# Verify image path and load image
img_path = '/content/drive/MyDrive/8710b98ea0_06E6522D6DINSPIRE-ortho.tif'

try:
    # Try loading with PIL
    pil_img = Image.open(img_path)
    print(f"Image loaded successfully. Size: {pil_img.size}, Mode: {pil_img.mode}")

    # Convert to OpenCV format (BGR) for YOLO
    img_cv = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)

except Exception as e:
    print(f"Error loading image: {e}")
    exit()

# Run inference
results = model(img_cv)  # Use the OpenCV format image

# Print raw results for debugging
print("\nRaw results:")
print(results.pandas().xyxy[0])  # Show detection dataframe

# Check if any detections were made
if len(results.pred[0]) == 0:
    print("\nNo objects detected in the image!")
    # Display original image for verification
    Image.fromarray(cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)).show()
    exit()

# Customize labels and display
detections = results.pred[0]
custom_labels = [model.names[int(det[5])] for det in detections]

# Set smaller font size
results.line_thickness = 2  # Smaller font size (default is 3)

# Render with custom labels
rendered_imgs = results.render(labels=[custom_labels])

# Convert to RGB and display
img_rgb = cv2.cvtColor(rendered_imgs[0], cv2.COLOR_BGR2RGB)
Image.fromarray(img_rgb).show()

# Optional: Save the result
Image.fromarray(img_rgb).save('detection_results.jpg')
print("Results saved as 'detection_results.jpg'")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-18 Python-3.11.13 torch-2.6.0+cu124 CPU

Fusing layers... 
Model summary: 322 layers, 86570425 parameters, 0 gradients, 205.0 GFLOPs
Adding AutoShape... 


Image loaded successfully. Size: (8408, 8217), Mode: RGBA


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):



Raw results:
           xmin         ymin         xmax         ymax  confidence  class  \
0   5001.030762  4085.542969  5669.449219  4890.261230    0.920580     48   
1   6237.523926  1903.156738  6712.354004  2643.826904    0.896793     48   
2   5623.293457  4699.416504  6527.979492  5180.741699    0.896147     48   
3   1037.878174  6390.415527  1483.748779  6944.252441    0.890179     48   
4   4159.068848  2249.600098  4460.300781  2695.617188    0.841216     48   
5   4546.161621  1479.830078  5162.719238  2035.968994    0.770562     48   
6   6258.726074  3399.744629  6756.489258  3732.276367    0.609113     48   
7   5866.694824  5101.972168  6194.458496  5296.350098    0.605234     48   
8   5861.940430  4024.758057  6732.298828  4427.846191    0.563029     48   
9   5762.722656  4339.859863  6627.028809  4781.377441    0.418175     27   
10  1915.072632  7766.327637  2629.892090  7945.369629    0.410362     48   
11  5942.531738  3672.568115  6826.865723  4097.902832    0.40

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Load and display the image
img = Image.open('/content/detection_results.jpg')  # Replace with your file path
plt.imshow(img)
plt.axis('off')  # Hide axes
plt.show()